<h1>Predicción con modelo Catboost</h1>

In [1]:
%pip install catboost
%pip install --upgrade pip
%pip install ipywidgets
%pip install ipython
%pip install catboost-widget

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement catboost-widget (from versions: none)
ERROR: No matching distribution found for catboost-widget


In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [32]:
dataframe = pd.read_csv("../data/clean_dataset_terror_movies.csv")
dataframe.head()

,Unnamed: 0.1,Unnamed: 0,id,adult,release_date,genre_ids,original_title,original_language,title,popularity,...,Acción,Comedia,Historia,Western,Suspense,Crimen,Ciencia ficción,Misterio,Documental,reciente
0,0,0,758323,False,2023-04-14,27-53,The Pope's Exorcist,en,El exorcista del papa,1630.483,...,0,0,0,0,1,0,0,0,0,0
1,1,1,980078,False,2023-02-15,27-53,Winnie the Pooh: Blood and Honey,en,Winnie the Pooh: Blood and Honey,996.295,...,0,0,0,0,1,0,0,0,0,0
2,2,2,1084225,False,2023-03-02,28-18-27-878-53,The Park,en,The Park,923.037,...,1,0,0,0,1,0,1,0,0,0
3,3,3,984105,False,2023-01-10,27,The Mummy Resurrection,en,The Mummy Resurrection,703.369,...,0,0,0,0,0,0,0,0,0,0
4,4,4,631842,False,2023-02-03,27-9648-53,Knock at the Cabin,en,Llaman a la puerta,687.168,...,0,0,0,0,1,0,0,1,0,0


In [33]:
dataframe = dataframe[["val_count",  "popularity", "vote_count",   'vote_average', 'budget', 'duration', 'revenues', 'spoken_languagess',  'quantile', 'count_labels', "is_terror_comedia", "reciente"]]
dataframe

KeyError: "['count_labels'] not in index"

In [5]:
#dataframe = dataframe[["id", "genre_list", "popularity"]]
#dataframe


In [6]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split


In [12]:
from catboost import CatBoostClassifier

#X, Y = make_multilabel_classification(n_samples=500, n_features=10, n_classes=5, random_state=0)
X = np.array(dataframe[["val_count",  "popularity", "vote_count",   'vote_average', 'budget', 'duration', 'revenues', 'spoken_languagess',  'quantile', 'count_labels']])
Y = np.array(dataframe['is_terror_comedia'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

#nombres_caracteristicas = X.columns.tolist()

# Definir el modelo
modelo = CatBoostClassifier(iterations=10, learning_rate=0.1, depth=2,  loss_function='MultiClass',eval_metric='Accuracy')


In [13]:
# Entrenar el modelo
modelo.fit(X_train, Y_train)

0:	learn: 0.7464000	total: 1.57ms	remaining: 14.2ms
1:	learn: 0.7464000	total: 3.07ms	remaining: 12.3ms
2:	learn: 0.7464000	total: 4.69ms	remaining: 10.9ms
3:	learn: 0.7464000	total: 6.14ms	remaining: 9.21ms
4:	learn: 0.8076000	total: 7.47ms	remaining: 7.47ms
5:	learn: 0.8076000	total: 9.02ms	remaining: 6.01ms
6:	learn: 0.8076000	total: 10.7ms	remaining: 4.57ms
7:	learn: 0.8374667	total: 12ms	remaining: 3ms
8:	learn: 0.8140000	total: 13.4ms	remaining: 1.49ms
9:	learn: 0.8374667	total: 14.8ms	remaining: 0us


In [30]:
importances = modelo.feature_importances_
feature_names = X.columns

# Imprimir los nombres de las características junto con sus importancias
for feature_name, importance in zip(feature_names, importances):
    print(feature_name, importance)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [14]:
# Depurar el modelo (15 iterations)
modelo.get_feature_importance(prettified=True)

,Feature Id,Importances
0,0,53.348727
1,8,25.294913
2,9,21.356360
3,1,0.000000
4,2,0.000000
5,3,0.000000
6,4,0.000000
7,5,0.000000
8,6,0.000000
9,7,0.000000


In [15]:
#Y_test =Y_test.reset_index(drop=True)

In [16]:
test_predict = modelo.predict(X_test)

In [17]:
accuracy = eval_metric(Y_test, test_predict, 'Accuracy')
print(f'Accuracy: {accuracy}')

Accuracy: [0.8372]


In [18]:
accuracy_per_class = eval_metric(Y_test, test_predict, 'Accuracy:type=PerClass')
for cls, value in zip(modelo.classes_, accuracy_per_class):
    print(f'Accuracy for class {cls}: {value}')

Accuracy for class 0: 0.8372


In [19]:
hamming = eval_metric(Y_test, test_predict, 'HammingLoss')[0]
print(f'HammingLoss: {hamming:.4f}')
mean_accuracy_per_class = sum(accuracy_per_class) / len(accuracy_per_class)
print(f'MeanAccuracyPerClass: {mean_accuracy_per_class:.4f}')
print(f'HammingLoss + MeanAccuracyPerClass = {hamming + mean_accuracy_per_class}')

HammingLoss: 0.1628
MeanAccuracyPerClass: 0.8372
HammingLoss + MeanAccuracyPerClass = 1.0


In [20]:
for metric in ('Precision', 'Recall', 'F1'):
    print(metric)
    values = eval_metric(Y_test, test_predict, metric)
    for cls, value in zip(modelo.classes_, values):
        print(f'class={cls}: {value:.4f}')
    print()

Precision
class=0: 0.9160

Recall
class=0: 0.6096

F1
class=0: 0.7321



In [21]:
# Definir el modelo
model = CatBoostClassifier(iterations=100, learning_rate=0.05, depth=6, eval_metric='Accuracy')


In [29]:
#X_train = X_train.reset_index(drop=True)

In [24]:
#Y_train =Y_train.reset_index(drop=True)


In [27]:
# Entrenar el modelo
model.fit(X_train, Y_train)#, eval_set=(X_test, Y_test))

0:	learn: 0.8657333	total: 3.73ms	remaining: 369ms
1:	learn: 0.8657333	total: 5.93ms	remaining: 291ms
2:	learn: 0.8666667	total: 7.92ms	remaining: 256ms
3:	learn: 0.8668000	total: 10.6ms	remaining: 254ms
4:	learn: 0.8660000	total: 12.9ms	remaining: 246ms
5:	learn: 0.8684000	total: 14.9ms	remaining: 233ms
6:	learn: 0.8724000	total: 16.9ms	remaining: 224ms
7:	learn: 0.8708000	total: 18.9ms	remaining: 218ms
8:	learn: 0.8694667	total: 20.6ms	remaining: 208ms
9:	learn: 0.8694667	total: 22.9ms	remaining: 206ms
10:	learn: 0.8698667	total: 25.4ms	remaining: 206ms
11:	learn: 0.8704000	total: 27.7ms	remaining: 203ms
12:	learn: 0.8730667	total: 29.9ms	remaining: 200ms
13:	learn: 0.8998667	total: 31.9ms	remaining: 196ms
14:	learn: 0.8997333	total: 34.1ms	remaining: 193ms
15:	learn: 0.8772000	total: 36.2ms	remaining: 190ms
16:	learn: 0.8709333	total: 38.1ms	remaining: 186ms
17:	learn: 0.8716000	total: 40.2ms	remaining: 183ms
18:	learn: 0.8842667	total: 42.7ms	remaining: 182ms
19:	learn: 0.9016000	t

In [28]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(Y_test, y_pred)
print("Precisión: {:.2f}%".format(accuracy * 100))

# Generar un informe de clasificación
report = classification_report(Y_test, y_pred)
print(report)


Precisión: 90.80%
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      1588
           1       0.86      0.89      0.88       912

    accuracy                           0.91      2500
   macro avg       0.90      0.90      0.90      2500
weighted avg       0.91      0.91      0.91      2500

